In [1]:
import sqlite3

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from daftpy.daftdata import get_db, db_dict, to_datetime, sale_dict_daily

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
plt.style.use('seaborn')

In [3]:
!ls data

2021-09-25.db  2021-09-30.db  2021-10-05.db  2021-10-10.db  2021-10-15.db
2021-09-26.db  2021-10-01.db  2021-10-06.db  2021-10-11.db  2021-10-16.db
2021-09-27.db  2021-10-02.db  2021-10-07.db  2021-10-12.db  2021-10-17.db
2021-09-28.db  2021-10-03.db  2021-10-08.db  2021-10-13.db  backup
2021-09-29.db  2021-10-04.db  2021-10-09.db  2021-10-14.db


------------------

-----------

In [4]:
sale_dict = db_dict()
sale_dict.keys()

dict_keys(['2021-10-05', '2021-09-29', '2021-10-16', '2021-10-09', '2021-10-04', '2021-10-01', '2021-09-28', '2021-09-27', '2021-10-06', '2021-10-15', '2021-10-03', '2021-09-30', '2021-10-11', '2021-10-08', '2021-10-12', '2021-10-02', '2021-10-14', '2021-10-13', '2021-09-26', '2021-10-10', '2021-10-17', '2021-10-07', '2021-09-25'])

In [5]:
sorted(sale_dict.keys())

['2021-09-25',
 '2021-09-26',
 '2021-09-27',
 '2021-09-28',
 '2021-09-29',
 '2021-09-30',
 '2021-10-01',
 '2021-10-02',
 '2021-10-03',
 '2021-10-04',
 '2021-10-05',
 '2021-10-06',
 '2021-10-07',
 '2021-10-08',
 '2021-10-09',
 '2021-10-10',
 '2021-10-11',
 '2021-10-12',
 '2021-10-13',
 '2021-10-14',
 '2021-10-15',
 '2021-10-16',
 '2021-10-17']

In [6]:
import collections

sale_dict = collections.OrderedDict(sorted(sale_dict.items())).copy()

sale_dict.keys()

odict_keys(['2021-09-25', '2021-09-26', '2021-09-27', '2021-09-28', '2021-09-29', '2021-09-30', '2021-10-01', '2021-10-02', '2021-10-03', '2021-10-04', '2021-10-05', '2021-10-06', '2021-10-07', '2021-10-08', '2021-10-09', '2021-10-10', '2021-10-11', '2021-10-12', '2021-10-13', '2021-10-14', '2021-10-15', '2021-10-16', '2021-10-17'])

Ahora haremos una sola tabla que contenga todos los datos que necesitamos. Como cada dia se escrapean todos los datos de la web, hay muchos anuncios que tendremos que eliminar. Queremos una tabla en la que aparezcan todos los anuncios nuevos de cada dia, pero no los renovados sin modificacion en el precio. 

Para obtener la tabla deseada tendremos que seguir los pasos siguientes:

1. Filtrar cada dataset de forma que obtengamos solo los anuncios cuyo `entered_renewed` se corresponda con el dia de escrapeo, `scraping_date`. De esta forma tenemos los anuncios renovados o publicados actualizados de cada dia.
2. Eliminar los anuncios renovados, ya que entendemos que no es un nuevo activo disponible para invertir en el. En caso de que el precio del activo cambie al ser renovado si se tendra en cuenta y no sera eliminado. En caso de que el precio del anuncio cambie al ser renovado, se eliminara el anuncio antiguo, ya que deja de reflejar el precio actual.

--------

En primer lugar usamos la funcion `to_datetime` del modulo `daftdata` para convertir las columnas `entered_renewed` y `scraping_date` a `datetime64` type.

In [7]:
to_datetime(sale_dict)

In [8]:
sale_dict['2021-09-25'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16368 entries, 0 to 16367
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   daft_id                       16368 non-null  object        
 1   item_id                       16368 non-null  object        
 2   url                           16368 non-null  object        
 3   name                          16368 non-null  object        
 4   price                         16368 non-null  object        
 5   info                          16368 non-null  object        
 6   sale_type                     16368 non-null  object        
 7   floor_area                    16368 non-null  object        
 8   psr                           16368 non-null  object        
 9   ber                           16368 non-null  object        
 10  entered_renewed               16368 non-null  datetime64[ns]
 11  views                       

In [9]:
(sale_dict['2021-09-25']['entered_renewed'].dt.day == sale_dict['2021-09-25']['scraping_date'].dt.day.value_counts().index[0]).sum()

2368

 ############################## Deberia coger todos los anuncios del primer dataset? o al menos mas dias?

Utilizamos la funcion `sale_dict_daily` para crear un dictionary que contenga en cada elemento la fecha de escrapeo (key) y los anuncios publicados o renovados ese dia.

In [10]:
daily_dict = sale_dict_daily(sale_dict)

In [11]:
twenty_five = daily_dict['2021-09-25']
print(twenty_five.shape)
twenty_six = daily_dict['2021-09-26']
print(twenty_six.shape)

(2368, 17)
(2339, 17)


In [12]:
twenty_five['url'].sample().values

array(['https://www.daft.ie/for-sale/semi-detached-house-34-an-sruth-n-turlough-road-castlebar-co-mayo/3511046'],
      dtype=object)

In [13]:
count = 0
for url in twenty_five['url']:
    if twenty_six['url'].str.contains(url).sum() != 0:
        count += 1
print(count)

2195


In [14]:
url = 'https://www.daft.ie/for-sale/semi-detached-house-43-rowan-drive-castlebar-co-mayo/3472240'
twenty_six[twenty_six['url'].str.contains(url)]#.sum()

,daft_id,item_id,url,name,price,info,sale_type,floor_area,psr,ber,entered_renewed,views,type_house,energy_performance_indicator,coordinates,type,scraping_date
1111,15734751,3472240,https://www.daft.ie/for-sale/semi-detached-hou...,"43 Rowan Drive, Castlebar, Co. Mayo","€120,000","3 Bed,1 Bath,81 m²,Semi-D",For Sale by Private Treaty,81 m²,none,E1,2021-09-26,"3,906",house,306.56 kWh/m2/yr,53.861236+-9.291636,buy,2021-09-26


In [15]:
(twenty_six[twenty_six['url'].str.contains(url)].values == twenty_five[twenty_five['url'].str.contains(url)].values).all(axis=1)[0]

False

In [16]:
twenty_six[twenty_six['url'].str.contains(url)].values

array([['15734751', '3472240',
        'https://www.daft.ie/for-sale/semi-detached-house-43-rowan-drive-castlebar-co-mayo/3472240',
        '43 Rowan Drive, Castlebar, Co. Mayo', '€120,000',
        '3 Bed,1 Bath,81 m²,Semi-D', 'For Sale by Private Treaty',
        '81 m²', 'none', 'E1', Timestamp('2021-09-26 00:00:00'), '3,906',
        'house', '306.56 kWh/m2/yr', '53.861236+-9.291636', 'buy',
        Timestamp('2021-09-26 00:00:00')]], dtype=object)

In [17]:
twenty_five[twenty_five['url'].str.contains(url)].values

array([['15734751', '3472240',
        'https://www.daft.ie/for-sale/semi-detached-house-43-rowan-drive-castlebar-co-mayo/3472240',
        '43 Rowan Drive, Castlebar, Co. Mayo', '€120,000',
        '3 Bed,1 Bath,81 m²,Semi-D', 'For Sale by Private Treaty',
        '81 m²', '004215', 'E1', Timestamp('2021-09-25 00:00:00'),
        '3,872', 'house', '306.56 kWh/m2/yr', '53.861236+-9.291636',
        'buy', Timestamp('2021-09-25 00:00:00')]], dtype=object)

In [18]:
twenty_six[twenty_six['url'].str.contains(url)].values == twenty_five[twenty_five['url'].str.contains(url)].values

array([[ True,  True,  True,  True,  True,  True,  True,  True, False,
         True, False, False,  True,  True,  True,  True, False]])

In [19]:
twenty_five[twenty_five['url'].str.contains(url)].values[0][[8, 10, 11, 16]]

array(['004215', Timestamp('2021-09-25 00:00:00'), '3,872',
       Timestamp('2021-09-25 00:00:00')], dtype=object)

In [20]:
twenty_six[twenty_six['url'].str.contains(url)].values[0][[8, 10, 11, 16]]

array(['none', Timestamp('2021-09-26 00:00:00'), '3,906',
       Timestamp('2021-09-26 00:00:00')], dtype=object)

In [21]:
twenty_six.loc[twenty_six['url'].str.contains(url), ['url', 'price']].values

array([['https://www.daft.ie/for-sale/semi-detached-house-43-rowan-drive-castlebar-co-mayo/3472240',
        '€120,000']], dtype=object)

In [22]:
def drop_renewed(old_data, new_data):
    print(f'Shape before dropping: {new_data.shape}')
    for url in new_data['url']:
    
        condition_1 = old_data['url'].str.contains(url).sum() != 0
        condition_2 = (new_data.loc[new_data['url'].str.contains(url), ['url', 'price']].values == 
                       old_data.loc[old_data['url'].str.contains(url), ['url', 'price']].values).all() # axis=1
    
        if condition_1 and condition_2:
            index_to_drop = new_data[new_data['url'].str.contains(url)].index[0]
            new_data.drop(index=[index_to_drop], inplace=True)
    print(f'Shape after dropping: {new_data.shape}')
    print('-'*10)
    return new_data

In [23]:
sale_dict.keys()

odict_keys(['2021-09-25', '2021-09-26', '2021-09-27', '2021-09-28', '2021-09-29', '2021-09-30', '2021-10-01', '2021-10-02', '2021-10-03', '2021-10-04', '2021-10-05', '2021-10-06', '2021-10-07', '2021-10-08', '2021-10-09', '2021-10-10', '2021-10-11', '2021-10-12', '2021-10-13', '2021-10-14', '2021-10-15', '2021-10-16', '2021-10-17'])

In [24]:
full_data = daily_dict['2021-09-25']
print(f'Initial shape: {full_data.shape}')
print('-'*30)
daily_dict.pop('2021-09-25')

for i, key in enumerate(daily_dict):
    #if key == '2021-09-29': ####
     #   break
    print(f'Key: {key}')
    data_to_concat = drop_renewed(full_data, daily_dict[key])
    full_data = pd.concat([data_to_concat, full_data], axis=0)
    print(f'Shape after concatenation {1}: {full_data.shape}')
    print('-'*20)
print(f'Final shape: {full_data.shape}')

Initial shape: (2368, 17)
------------------------------
Key: 2021-09-26
Shape before dropping: (2339, 17)
Shape after dropping: (144, 17)
----------
Shape after concatenation 1: (2512, 17)
--------------------
Key: 2021-09-27
Shape before dropping: (2351, 17)
Shape after dropping: (169, 17)
----------
Shape after concatenation 1: (2681, 17)
--------------------
Key: 2021-09-28
Shape before dropping: (2275, 17)
Shape after dropping: (106, 17)
----------
Shape after concatenation 1: (2787, 17)
--------------------
Key: 2021-09-29
Shape before dropping: (2283, 17)
Shape after dropping: (112, 17)
----------
Shape after concatenation 1: (2899, 17)
--------------------
Key: 2021-09-30
Shape before dropping: (2065, 17)
Shape after dropping: (183, 17)
----------
Shape after concatenation 1: (3082, 17)
--------------------
Key: 2021-10-01
Shape before dropping: (2133, 17)
Shape after dropping: (267, 17)
----------
Shape after concatenation 1: (3349, 17)
--------------------
Key: 2021-10-02
Sha

In [25]:
full_data.shape

(6573, 17)